In [4]:
import numpy as np
from numba import cuda

In [5]:
@cuda.jit
def double_elements(src, dst):
    # Get the thread position in the grid
    tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    
    # Make sure we don't go out of bounds
    if tid < src.size:
        # Map function f(x) = 2x
        dst[tid] = 2 * src[tid]

def main():
    src_array = np.array(
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
        dtype=np.int32
    )
    print("Input array:", src_array)
    
    # CPU to GPU
    d_src = cuda.to_device(src_array)
    d_dst = cuda.device_array_like(d_src)
    
    # CPU ask GPU
    block_size = 256  # threads per block
    grid_size = (src_array.size + block_size - 1) // block_size
    
    # GPU with kernel
    double_elements[grid_size, block_size](d_src, d_dst)
    cuda.synchronize()
    
    # GPU to CPU
    result = d_dst.copy_to_host()
    print("Output array:", result)

if __name__ == '__main__':
    main()

Input array: [ 1  2  3  4  5  6  7  8  9 10]
Output array: [ 2  4  6  8 10 12 14 16 18 20]


/home/ctl6998/miniconda3/envs/hpc/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
